In [2]:
import numpy as np
import pandas as pd
import zipfile

zfile = zipfile.ZipFile('/content/attacked_messages.zip','r')
for filename in zfile.namelist():
  data = zfile.read(filename)
  file = open(filename, 'w+b')
  file.write(data)
  file.close() 

In [3]:
data_train = []
data_test = []
test_file = '/content/test.csv'

train_list = ['aircraft_28_3c09ee_attacked.csv',
              'aircraft_31_3c0fe6_attacked.csv',
              'aircraft_33_3c4907_attacked.csv',
              'aircraft_35_3c49eb_attacked.csv',
              'aircraft_36_3c56e6_attacked.csv',
              'aircraft_37_3c5ee1_attacked.csv',
              'aircraft_38_3c6441_attacked.csv',
              'aircraft_45_3c6624_attacked.csv',
              'aircraft_48_3c6676_attacked.csv',
              'aircraft_49_3c66a5_attacked.csv'
              ]
test_list = ['aircraft_0_00a2e4_attacked.csv',
             'aircraft_20_394c12_attacked.csv',
             'aircraft_23_3991e2_attacked.csv',
             'aircraft_24_39bd27_attacked.csv'
             ]

In [4]:
for tr in train_list:
  df1 = pd.read_csv(tr)
  data_train.append(df1)

for te in train_list:
  df2 = pd.read_csv(te)
  data_test.append(df2)

In [5]:
df_train = pd.concat(data_train)
df_test = pd.concat(data_test)

In [6]:
len(df_train)

66172

In [26]:
class DataLoader():
    def __init__(self,dataset):
        self.data_train = dataset
        self.len_train  = len(self.data_train)
        self.len_train_windows = None

    def get_data(self, seq_len):
        data_x = []
        data_y = []
        for i in range(self.len_train - seq_len):
            x, y = self._next_window(i, seq_len)
            data_x.append(x)
            data_y.append(y)
        return np.array(data_x), np.array(data_y)

    def _next_window(self, i, seq_len):
        window = self.data_train[i:i+seq_len]
        x = window[:-1]
        y = window[-1, [0]]
        return x, y

In [27]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
train_df = min_max_scaler.fit_transform(df_train)
test_df = min_max_scaler.fit_transform(df_test)

In [94]:
data = DataLoader(train_df)
x_train, y_train = data.get_data(seq_len=6)

In [95]:
n_features = train_df.shape[1]

In [121]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import EarlyStopping

In [122]:
class Model():
  def __init__(self):
    self.model = Sequential()

  def lstm_model(self):
    self.model.add(LSTM(64, input_shape=(5, n_features), return_sequences=True))
    self.model.add(Dropout(0.2))
    self.model.add(LSTM(32,return_sequences=True))
    self.model.add(Dropout(0.2))
    self.model.add(Dense(1,  kernel_initializer='glorot_uniform',activation='sigmoid'))
    self.model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    self.model.summary()
    
  def train(self, x, y, epochs, batch_size):
    print('[Model] Training Started')

    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
    
    self.model.fit(
			x,
			y,
			epochs=epochs,
			batch_size=batch_size,
			callbacks=callbacks
		)


In [123]:
model = Model()
model.lstm_model()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_40 (LSTM)              (None, 5, 64)             18432     
                                                                 
 dropout_32 (Dropout)        (None, 5, 64)             0         
                                                                 
 lstm_41 (LSTM)              (None, 5, 32)             12416     
                                                                 
 dropout_33 (Dropout)        (None, 5, 32)             0         
                                                                 
 dense_18 (Dense)            (None, 5, 1)              33        
                                                                 
Total params: 30,881
Trainable params: 30,881
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.train(x_train, y_train,
		epochs =10 ,
		batch_size =64
	)

[Model] Training Started
Epoch 1/10
1034/1034 [==============================] - 16s 11ms/step - loss: 0.0072 - accuracy: 1.1486e-04
Epoch 2/10
1034/1034 [==============================] - 12s 12ms/step - loss: 9.1326e-04 - accuracy: 9.0681e-05
Epoch 3/10
1034/1034 [==============================] - 12s 11ms/step - loss: 7.7102e-04 - accuracy: 9.0681e-05
Epoch 4/10
1034/1034 [==============================] - 12s 11ms/step - loss: 6.7841e-04 - accuracy: 9.0681e-05
Epoch 5/10
1034/1034 [==============================] - 11s 11ms/step - loss: 5.8908e-04 - accuracy: 9.0681e-05
Epoch 6/10
1034/1034 [==============================] - 11s 11ms/step - loss: 5.3336e-04 - accuracy: 9.0681e-05
Epoch 7/10
 336/1034 [========>.....................] - ETA: 7s - loss: 4.9019e-04 - accuracy: 1.3951e-04

KeyboardInterrupt: ignored